In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import zipfile
from zipfile import ZipFile

In [ ]:
file_name = "data/moving2.zip"

In [ ]:
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()

In [ ]:
#Importing the csv file created in factory io
data = pd.read_csv('data/moving2.zip')
df = pd.DataFrame(data)

In [ ]:
#Check data is imported
df.head(20)

In [ ]:
#Subseting all Light Array Emitter data
#using a copy to prevent index warning
filteredLAE = df[df[' Name']=='Light Array Emitter 1 (Value)']
#converting the current value column to numeric
filteredLAE = filteredLAE.copy()
filteredLAE[' Current Value'] = pd.to_numeric(filteredLAE[' Current Value'])

In [ ]:
filteredLAE[' Sensor Value Difference'] = filteredLAE[' Current Value'].diff()
filteredLAE

In [ ]:
#filteredLAE.to_csv (r'C:\Users\a.sethu\Desktop\Python\export.csv', index = False, header=True)

In [ ]:
#Filtering sensor trigger data points from the data by removing 
#all all data with current value = 0
filteredLAE = filteredLAE[filteredLAE[' Sensor Value Difference'] != 0]
filteredLAE = filteredLAE[filteredLAE[' Current Value']!=128 ]
filteredLAE = filteredLAE[filteredLAE[' Current Value']!=64 ]
#Dropping al NaN values
filteredLAE.dropna(subset = [' Sensor Value Difference'], inplace=True)
filteredLAE

In [ ]:
filteredLAE=filteredLAE.copy()
filteredLAE[filteredLAE[' Sensor Value Difference'] > 0]
filteredLAE['Trigger Status']="Off"
filteredLAE['Trigger Status'] = filteredLAE.apply(lambda x: 'On' if x[' Sensor Value Difference']>0 else 'Off',axis=1)

In [ ]:
filteredLAE['Trigger Interval'] = filteredLAE[" Time Elapsed"].diff(1)

In [ ]:
preprocessedDF = filteredLAE[filteredLAE['Trigger Status']=='On']

In [ ]:
preprocessedDF = preprocessedDF.copy()
preprocessedDF['Trigger Interval'] = preprocessedDF[" Time Elapsed"].diff(1)
preprocessedDF.dropna(subset = ['Trigger Interval'], inplace=True)
preprocessedDF

In [ ]:
import seaborn as sns
preprocessedDF=preprocessedDF.reset_index(drop=True)
plt.figure(figsize=(12,8)) #Plotting the data points on the plot
sns.stripplot(x='Trigger Interval',data = preprocessedDF,jitter= True,edgecolor = 'None')
sns.despine
locs,labels = plt.xticks()
plt.yticks([])

In [ ]:
import jenkspy
#Finding the breaks using the Jenks natural breaks optimzation method
breaks = jenkspy.jenks_breaks(preprocessedDF['Trigger Interval'],nb_class = 3)

In [ ]:
#show ranges
breaks

In [ ]:
#Box plot
import numpy as np
sns.boxplot(x='Trigger Interval', data=preprocessedDF, whis=np.inf)

In [ ]:
#All code below this is tp be ignored was used only for research purposes
#Removing data outside the range of3.8 to 8.7 for testing purposes
dumydata = preprocessedDF[preprocessedDF['Trigger Interval']>3.8]
dumydata = dumydata[dumydata['Trigger Interval']<8.7]

In [ ]:
dummydata=dummydata.reset_index(drop=True)
import jenkspy
#Finding the breaks using the Jenks natural breaks optimzation method
breaks = jenkspy.jenks_breaks(dummydata['Trigger Interval'],nb_class = 3)
#seaborn library for plotting
import seaborn as sns
plt.figure(figsize=(12,8)) #Plotting the data points on the plot
sns.stripplot(x='Trigger Interval',data = dummydata,jitter= True,edgecolor = 'None')
sns.despine
locs,labels = plt.xticks()
plt.yticks([])
#Drawing vertical lines usng the data points as x values
for b in breaks:
    plt.vlines(b,ymin= -0.2,ymax=0.2)